<a href="https://colab.research.google.com/github/repitta/CienciaDeDadosEducacionais/blob/master/notas_das_listas_de_LOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# These are the library we are going to use
import pandas as pd


In [0]:
# Situções finais no siga
# Reading the data directly from the Git
situacoes_url = ["https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2017.2.csv",
                      "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2018.1.csv",
                      "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2018.2.csv",
                      "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2019.1.csv",
                      "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lopSituacao2019.2.csv"]
usuarios_lop_url   =  "https://raw.githubusercontent.com/repitta/CienciaDeDadosEducacionais/master/arquivosCSV/lop_db_2019_09_csv_tratados/lopUsuarioTurma%20(1).csv"
 
submissoes_url     =  "https://github.com/repitta/CienciaDeDadosEducacionais/raw/master/arquivosCSV/lop_db_2019_09_csv_tratados/lopSubLista%20(1).csv"

In [0]:
# Situações finais since 2017.2 until 2019.2 
situacoes2017_2 = pd.read_csv(situacoes_url[0], sep=',')
situacoes2018_1 = pd.read_csv(situacoes_url[1], sep=',')
situacoes2018_2 = pd.read_csv(situacoes_url[2], sep=',')
situacoes2019_1 = pd.read_csv(situacoes_url[3], sep=',')
situacoes2019_2 = pd.read_csv(situacoes_url[4], sep=',')

# Submissoes dos alunos no LOP

submissoes = pd.read_csv(submissoes_url, sep=',')

#Usuarios cadastrados no LOP 
usuarios_lop = pd.read_csv(usuarios_lop_url, sep=',')

In [0]:
# Setting priginal data into one vector due to easier manipulation
situacoes =[situacoes2017_2,situacoes2018_1, situacoes2018_2, situacoes2019_1, situacoes2019_2]

In [0]:
# Eliminar a coluna Unnamed: 0 de todas as tabelas
for i in range(5):
  situacoes[i] = situacoes[i].drop(columns="Unnamed: 0")
  situacoes[i].fillna(0, inplace=True)
  situacoes[i]["matricula"] = situacoes[i]["matricula"].astype(str)
  situacoes[i]= situacoes[i].rename(columns={"ano": "anoSigaa"})
  situacoes[i]["anoSigaa"] = situacoes[i]["anoSigaa"].astype(float)
  situacoes[i]["turma"] = situacoes[i]["turma"].astype(str)
  
  
submissoes   = submissoes.drop(columns="Unnamed: 0")
submissoes.fillna(0, inplace=True)
submissoes["data"] = pd.to_datetime(submissoes["data"], format = "%Y/%m/%dT%H:%M:%S")
submissoes = submissoes.rename(columns={"quantidade": "quantidadeTotalLista"})

usuarios_lop = usuarios_lop.drop(columns="Unnamed: 0")
usuarios_lop.fillna(0, inplace=True)
#tratar a coluna descricaoTurma da tabela usuarios_lop para ficar igual a coluna turma da tabela situações 
usuarios_lop = usuarios_lop.rename(columns={"descricaoTurma": "turma"})
usuarios_lop["turma"] =  usuarios_lop["turma"].str.split(expand=True)[5]

usuarios_lop["matricula"] = usuarios_lop["matricula"].astype(str)
usuarios_lop= usuarios_lop.rename(columns={"ano": "anoLop"})
usuarios_lop["anoLop"] = usuarios_lop["anoLop"].astype(float)
usuarios_lop["turma"] = usuarios_lop["turma"].astype(str)




In [6]:
print(situacoes[0].columns)
print(" ")
print(usuarios_lop.columns)
print(" ")
print(submissoes.columns)

Index(['matricula', 'situacao', 'anoSigaa', 'turma'], dtype='object')
 
Index(['idTurma', 'anoLop', 'turma', 'matricula', 'idAluno', 'nome'], dtype='object')
 
Index(['data', 'idAluno', 'idLista', 'porcentagemAcerto',
       'quantidadeTotalLista', 'questao', 'titulo'],
      dtype='object')


In [7]:
#juntar todas as tabelas do sigaa para juntar com a tabela de usuarios do LOP
a =  pd.concat([situacoes[0], situacoes[1], situacoes[2], situacoes[3],situacoes[4]], ignore_index=True, axis=0)

mergeSituacoeUsuarios = pd.merge(right= a, left = usuarios_lop, on=["matricula","turma"], how="outer")

#Limpar a tabela para ficar só com as ocorrencias de usuarios que estão cadastrados no Lop
mergeSituacoeUsuarios = mergeSituacoeUsuarios.loc[mergeSituacoeUsuarios["anoSigaa"] == mergeSituacoeUsuarios["anoLop"] ]
mergeSituacoeUsuarios.head(1)


,idTurma,anoLop,turma,matricula,idAluno,nome,situacao,anoSigaa
0,598e15296d8650eb27d52e3d,2017.2,01A,20170106538,599b051d495fac076a06d3f5,Aleff Ramon da Silva Costa,APROVADO,2017.2


In [0]:
#criar um campo extra na tabela submissoes para ter certeza que a submisao pertence ao aluno matriculado naquele semestre
def anosub(data):
  if data.month >= 8:
    anoSubmissao = data.year + 0.2
  else :
    anoSubmissao = data.year + 0.1
  return anoSubmissao
submissoes["anoSubmissao"] = submissoes["data"].apply(anosub)



In [9]:
#juntas as tabelas situacaoUsuario com a tabela de submissoes
mergeSituacoeUsuariosSubLista = pd.merge(right=mergeSituacoeUsuarios, left = submissoes, on=["idAluno"], how="outer")

#Remover os lixos... Onde o ano da Submissão não for igual ao ano da matricula no sigaa
#Limpar a tabela para ficar só com as ocorrencias de usuarios que são do mesmo ano da submissao e do sigaa
mergeSituacoeUsuariosSubLista = mergeSituacoeUsuariosSubLista.loc[mergeSituacoeUsuariosSubLista["anoSigaa"] == mergeSituacoeUsuariosSubLista["anoSubmissao"] ]
mergeSituacoeUsuariosSubLista.head(1)

,data,idAluno,idLista,porcentagemAcerto,quantidadeTotalLista,questao,titulo,anoSubmissao,idTurma,anoLop,turma,matricula,nome,situacao,anoSigaa
132,2017-08-14 16:29:10.522,5991cb910be9d42f2f68b906,59c859ad3ccb990189c845ac,100.0,15.0,59885f8d684f4a3f78163c00,Lista de Exercícios 1 - Expressões e variáveis,2017.2,598e15296d8650eb27d52e3d,2017.2,01A,20170039453,Amanda,APROVADO,2017.2


**Porcentagem Máxima por aluno em cada questão da lista**

In [10]:
NotaMaxQuestao = mergeSituacoeUsuariosSubLista.groupby(["idLista","idAluno","questao","anoSubmissao"]).max()
NotaMaxQuestao.rename(columns={"porcentagemAcerto":"porcentagemMaxima"}, inplace=True)
NotaMaxQuestao = NotaMaxQuestao.reset_index()
print(NotaMaxQuestao.columns)
NotaMaxQuestao.head(1)

Index(['idLista', 'idAluno', 'questao', 'anoSubmissao', 'data',
       'porcentagemMaxima', 'quantidadeTotalLista', 'titulo', 'idTurma',
       'anoLop', 'turma', 'matricula', 'nome', 'situacao', 'anoSigaa'],
      dtype='object')


,idLista,idAluno,questao,anoSubmissao,data,porcentagemMaxima,quantidadeTotalLista,titulo,idTurma,anoLop,turma,matricula,nome,situacao,anoSigaa
0,599ae0e1872c0a078b68b13e,5991cb910be9d42f2f68b906,5998737a54b78701f54e9666,2017.2,2017-08-23 18:19:58.885,100.0,7.0,Lista de Laboratório 2 - Expressões Lógicas e ...,598e15296d8650eb27d52e3d,2017.2,01A,20170039453,Amanda,APROVADO,2017.2


**Quantidade de questoes feitas por aluno em cada lista**

In [11]:
quantidade = NotaMaxQuestao.groupby(["titulo","idAluno","anoSubmissao"]).count()
quantidade.rename(columns={"quantidadeTotalLista":"questoesFeitas"}, inplace=True)
quantidade= quantidade.reset_index()
quantidade= quantidade[["titulo","idAluno","anoSubmissao","questoesFeitas"]]
print(quantidade.columns)
quantidade.head(1)

Index(['titulo', 'idAluno', 'anoSubmissao', 'questoesFeitas'], dtype='object')


,titulo,idAluno,anoSubmissao,questoesFeitas
0,Lista 1.0 - Expressões aritméticas,59b91272570dda40b535c50d,2019.2,10


**Nota final dos alunos por lista**

In [12]:
NotaLista = NotaMaxQuestao.groupby(["titulo","idAluno","anoSubmissao"]).sum()
NotaLista.rename(columns={"porcentagemMaxima":"SomaLista"}, inplace=True)
#NotaLista = NotaLista.drop('ano',axis=1)
NotaLista = NotaLista.reset_index()
NotaLista = NotaLista[["titulo",'idAluno',"anoSubmissao",'SomaLista']]
print(NotaLista.columns)
NotaLista.head(1)

Index(['titulo', 'idAluno', 'anoSubmissao', 'SomaLista'], dtype='object')


,titulo,idAluno,anoSubmissao,SomaLista
0,Lista 1.0 - Expressões aritméticas,59b91272570dda40b535c50d,2019.2,1000.0


In [13]:
mergeQuanNotaLista =  pd.merge(right= quantidade, left = NotaLista, on=['idAluno','titulo',"anoSubmissao"], how="inner")
print(mergeQuanNotaLista.columns)
print(mergeQuanNotaLista.shape)
mergeQuanNotaLista.head(1)

Index(['titulo', 'idAluno', 'anoSubmissao', 'SomaLista', 'questoesFeitas'], dtype='object')
(7216, 5)


,titulo,idAluno,anoSubmissao,SomaLista,questoesFeitas
0,Lista 1.0 - Expressões aritméticas,59b91272570dda40b535c50d,2019.2,1000.0,10


**Quantidade de questoes por lista**

In [14]:
quantQuesList = mergeSituacoeUsuariosSubLista[['quantidadeTotalLista','titulo']]
quantQuesList =quantQuesList.drop_duplicates()
print(quantQuesList.shape)
quantQuesList

(22, 2)


,quantidadeTotalLista,titulo
132,15.0,Lista de Exercícios 1 - Expressões e variáveis
136,9.0,Lista de Revisão Unidade 1
137,7.0,Lista de Laboratório 2 - Expressões Lógicas e ...
140,15.0,Lista de Exercícios 2 - Estruturas Condicionais
154,15.0,Lista de Exercícios 3 - Laço Condicional
156,7.0,Lista de Laboratório 3 - Estruturas de Repeti...
167,5.0,Prova 1 - Turma 09 - 2017.1
176,16.0,Lista de Exercícios 4 - Laço contado
179,5.0,Lista de Laboratório 4 - Repetição Contada
193,16.0,Lista de Exercícios 5 - Vetores


**Concatenando as informações das tabelas mergeQuanNotaLista e quantidade de questoes por lista para calcular o nota final de cada lista**

In [15]:
mergeQuanNotaListaQuanQ = mergeQuanNotaLista.join(quantQuesList.set_index('titulo'), on='titulo')
mergeQuanNotaListaQuanQ["NotaLista"]= mergeQuanNotaListaQuanQ["SomaLista"] /mergeQuanNotaListaQuanQ['quantidadeTotalLista']
print(mergeQuanNotaListaQuanQ.columns)
print(mergeQuanNotaListaQuanQ.shape)
mergeQuanNotaListaQuanQ.head(3)

Index(['titulo', 'idAluno', 'anoSubmissao', 'SomaLista', 'questoesFeitas',
       'quantidadeTotalLista', 'NotaLista'],
      dtype='object')
(7216, 7)


,titulo,idAluno,anoSubmissao,SomaLista,questoesFeitas,quantidadeTotalLista,NotaLista
0,Lista 1.0 - Expressões aritméticas,59b91272570dda40b535c50d,2019.2,1000.0,10,10.0,100.0
1,Lista 1.0 - Expressões aritméticas,5b7b1f4f9513a60408ac2d21,2019.2,1000.0,10,10.0,100.0
2,Lista 1.0 - Expressões aritméticas,5c77f58d7776520411e9aff4,2019.1,1000.0,10,10.0,100.0


In [16]:
mergeSituacoeUsuariosSubLista.columns

Index(['data', 'idAluno', 'idLista', 'porcentagemAcerto',
       'quantidadeTotalLista', 'questao', 'titulo', 'anoSubmissao', 'idTurma',
       'anoLop', 'turma', 'matricula', 'nome', 'situacao', 'anoSigaa'],
      dtype='object')

**Tabela final com todas as informações do aluno das listas**

In [17]:
mergeSituacoeUsuariosSubListaM = mergeSituacoeUsuariosSubLista[['idAluno','titulo', 'anoSubmissao','anoLop', 'turma', 'matricula', 'nome', 'situacao', 'anoSigaa']]
tabelaFinalListas= mergeQuanNotaListaQuanQ.join(mergeSituacoeUsuariosSubListaM.set_index(['idAluno','titulo', 'anoSubmissao']), on=['idAluno','titulo', 'anoSubmissao'])
#remover as linhas duplicadas
tabelaFinalListas = tabelaFinalListas.drop_duplicates()
tabelaFinalListas.head()


,titulo,idAluno,anoSubmissao,SomaLista,questoesFeitas,quantidadeTotalLista,NotaLista,anoLop,turma,matricula,nome,situacao,anoSigaa
0,Lista 1.0 - Expressões aritméticas,59b91272570dda40b535c50d,2019.2,1000.0,10,10.0,100.0,2019.2,03A,2016012150,João,MATRICULADO,2019.2
1,Lista 1.0 - Expressões aritméticas,5b7b1f4f9513a60408ac2d21,2019.2,1000.0,10,10.0,100.0,2019.2,03D,20180034595,Eduardo Freire de Almeida,MATRICULADO,2019.2
2,Lista 1.0 - Expressões aritméticas,5c77f58d7776520411e9aff4,2019.1,1000.0,10,10.0,100.0,2019.1,02D,20180146250,Vitor da Rocha Silva,REPROVADO,2019.1
3,Lista 1.0 - Expressões aritméticas,5c77fa897776520411e9b01c,2019.2,1000.0,10,10.0,100.0,2019.2,02C,20180044170,Wendson Canuto Bezerra Dos Santos,MATRICULADO,2019.2
4,Lista 1.0 - Expressões aritméticas,5c87bd6923798d040d0270ad,2019.2,1000.0,10,10.0,100.0,2019.2,03D,20180135195,José Humberto Valente da Silva Junior,MATRICULADO,2019.2


**Exportar a tabela final das notas das listas para um arquivo CSV**

In [0]:
 tabelaFinalListas.to_csv(r'notasListas.csv')

In [0]:
def tabelaNotaLista(ano, titulo, turma): 
  tabela = tabelaFinalListas.loc[ (tabelaFinalListas["anoSigaa"] == ano) & (tabelaFinalListas["titulo"].str.contains(titulo)) & (tabelaFinalListas["turma"].str.contains(turma)) ,: ]
  tabela = tabela[["anoSigaa","matricula","turma", "nome", "NotaLista","SomaLista"]].sort_values(by="nome")
  return tabela

**Exportando o dataframe para um arquivo excel**

In [0]:
turmas= ["03A","03B","03C","03D"]
listas = ["Lista 1.2","Estruturas Condicionais"]
for t in turmas:
  for l in listas:
    tab = tabelaNotaLista(2019.2, l, t)
    tab =tab[["anoSigaa","matricula","turma", "nome", "NotaLista"]].sort_values(by="nome")
    tab.to_excel(r'Nota'+l+" "+t+'.xls')

In [0]:
#Fiz essa lista separa pq o professor não cobrou todas as questoes para valer nota .. apenas 10 questoes das 15 que tem na lista
turmas= ["03A","03B","03C","03D"]
for t in turmas:
    tab = tabelaNotaLista(2019.2, "3 - La", t)
    tab["NotaLista"] = tab["SomaLista"] / 10
    tab =tab[["anoSigaa","matricula","turma", "nome", "NotaLista"]].sort_values(by="nome")
    tab.to_excel(r'NotaLista03'+" "+t+'.xls')